# Notebook 3: NB3-OceanSim

# Coupled ocean circulation wave model on real topography


The objective of this notebook is to use a coupled ocean/wave circulation model to investigate impact of wind direction and speed on ocean current at regional scale.

The  model, we will be using, is a modified version of <a href='http://swanmodel.sourceforge.net'>SWAN</a> which has been softly coupled to a modified version of tawic (from <a href='http://www.sciencedirect.com/science/article/pii/S0098300496000477'>F. J. Caviglia & W. C. Dragani, 1996</a>). The model runs in **stationary** mode which means that it outputs condition of ocean currents and waves for a given wind field after convergence of the numerical solution. This kind of model is usefull when looking at long-term trends in ocean evolution over regional scale (such as climate change science or surface process modelling at geological scale). 

# Swan wave model

SWAN is a third-generation wave model, developed at Delft University of Technology, that computes random, short-crested wind-generated waves in coastal regions and inland waters. For more information about SWAN, see a short overview of <a href='http://swanmodel.sourceforge.net/features/features.htm'>model features</a>. 

### SWAN Hindcast tidal inlet Eastern Scheldt, the Netherlands
<img src='http://www.svasek.com/img/sv1461-easternscheldt.gif' width=500>



# Tawic mode

Tawic computes tide- and wind-driven currents by solving the nonlinear two-dimensional hydrodynamic equations using a finite differencing method. For more information about Tawic read the descrption <a href='http://www.sciencedirect.com/science/article/pii/S0098300496000477'>paper</a>.

### Circulation in Massachusetts bay﻿
<img src='https://lh3.googleusercontent.com/-cw3W6wOSGE4/U7qD50DCGOI/AAAAAAAAAUw/jK7mHtQMTlc/w1012-h962/massachusets_ocean.png' width=500>

<br/>
<br/>

The data manipulation preserves the georeferencing information of the DEM, allowing later projection of the model in a GIS client (or Google Earth).
First we upload the libraries that we will use...

In [1]:
%pylab inline
%matplotlib inline
import os
import csv
import glob
import subprocess
import numpy as np
import myOSshell as myos

Populating the interactive namespace from numpy and matplotlib


# Model setup

We will test the coupled model on the South Australian shelf around Kangaroo Island.

I've been using the same XYZ ASCII files from GA that we've manipulated on previous notebook.

<small>*Reference: Whiteway, T., 2009. Australian Bathymetry and Topography Grid, June 2009. Scale 1:5000000. Geoscience Australia, Canberra. http://dx.doi.org/10.4225/25/53D99B6581B9A*
</small>

In [12]:
import gmaps
data = [ [ -35.8333,137.2500],[-30.7678,128.9700] ,[-37.8000, 144.950]]
gmaps.heatmap(data, width="400px")

The initial grid looks like this:
<img src='./gab/gab2.jpg' width=800>

# Input files


The dataset for running the example is located in the **gab** repository and consist in:

+ a input file name gabocean.xml
+ a **data** folder containing the grid information *gab_topo.nodes* which has the same format as the one we've just created for Sydney area

Let us have a look at the **input file: gabocean.xml**

In [7]:
%ls
myos.list_dir('./gab')

bath.csv  gab/  Lecture-overview.ipynb  myOSshell.py  myOSshell.pyc  NB1-SWE2D.ipynb  NB2-BathymetryGen.ipynb  NB3-OceanSim.ipynb  SydOceanT/  XYZ_Index.jpg
['./gab/gabocean.xml', './gab/data', './gab/gab2.jpg', './gab/gab1.jpg', './gab/gab3.jpg']


In [ ]:
# name: gabocean.xml
# type: text/xml
# size: 4266 bytes 
###########################
<?xml version="1.0" encoding="UTF-8"?>
<lecode:oceanInput xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
xmlns:lecode="http://subversion.assembla.com/svn/xsd/XSDlecode"
xsi:schemaLocation="http://subversion.assembla.com/svn/xsd/XSDlecode
http://subversion.assembla.com/svn/xsd/XSDlecode/OCEAN.xsd">

<!-- Declaration of the DEM file -->
<TopoGrid>
    <!-- Name of the DEM file -->
    <Grid>data/gab_topo.nodes</Grid>
    <!-- Number of nodes along the X and Y direction -->
    <GridX>250</GridX>
    <GridY>250</GridY>
    <!-- Grid spacing in metres --> 
    <GridSpace>3000</GridSpace>
    <!-- Mean latitude of the grid -->
    <latGrid>-32.5</latGrid>
    <!-- Synchronisation folder if coupled to sediment transport model -->
    <syncFolder>Poseidon</syncFolder>
    <!-- Flag for sediment transport model : 0=off - 1=on -->
    <spmForcing>0</spmForcing>
    <!-- Top sedimentary layer thickness (in metres)-->
    <initDep>50</initDep>
    <!-- Flag for ocean circulation model: 0=off - 1=on -->
    <circForcing>1</circForcing>
    <!-- Flag for wave  model: 0=off - 1=on -->
    <waveForcing>1</waveForcing>
</TopoGrid>
  
<!-- Declaration of simulation time -->
<Time>
    <!-- Start time in years -->
    <startTime>0</startTime>
    <!-- End time in years -->
    <endTime>0.2</endTime>
    <!-- Synchronisation time in years when coupling to sediment transport model -->
    <syncTime>0.1</syncTime>
    <!-- Output time in years -->
    <outputTime>0.1</outputTime>
</Time>
  
<!-- Declaration of ocean circulation parameters -->
<CirculationParam>
    <!--  Time step limitation (Courant numbers) Calculations usually become unstable for Courant numbers > 3-5.-->
    <Courant>1</Courant>
    <!-- Storm averaged duration in hours -->  
    <stormTime>48</stormTime>
    <!-- Coefficient of bottom friction -->
    <fricCoef>0.0005</fricCoef>
     <!-- number of time-steps at which the calculated values of U, V, and surface elevation are filtered--> 
    <filterStep>10</filterStep>
</CirculationParam>
  
<!-- Declaration of wave and wind forcing parameters -->
  <OceanForecast>
    <!-- Maximum water depth (metres) above which waves have no effect -->
    <waveBase>50.0</waveBase>
    <!-- Wind scenarios number: 
    you will need to define as many forecastClass elements as the number of forecast defined here-->
    <nbForecast>2</nbForecast>
    <forecastClass>
       <!-- Each forecast can have several wind parameters subgroups 
        This element defines the number of subgroups -->
    <subgroupNb>1</subgroupNb>
       <!-- Start time of the forecast in years -->
    <start>0</start>
       <!-- End time of the forecast in years -->
    <end>0.1</end>
       <!-- Subgroup wind parameters which consists in:
        - first number: percentage of activity (between 0 and 1) of the considered subgroup over the considered forecast time only relevant if sediment transport is turned on
        - second number: wind speed in m/s at 10 m above sea-level
        - third number direction towards which the wind is blowing (in degrees) starting from the Ox axis
    -->
      <subgroupParam>0.3 20.0 140.0</subgroupParam>
    </forecastClass>
    <forecastClass>
       <!-- Each forecast can have several wind parameters subgroups. 
        This element defines the number of subgroups. -->
    <subgroupNb>1</subgroupNb>
       <!-- Start time of the forecast in years -->
    <start>0.1</start>
       <!-- End time of the forecast in years -->
    <end>0.2</end>
       <!-- Subgroup wind parameters which consists in:
        - first number: percentage of activity (between 0 and 1) of the considered subgroup over the considered forecast time only relevant if sediment transport is turned on
        - second number: wind speed in m/s at 10 m above sea-level
        - third number direction towards which the wind is blowing (in degrees) starting from the Ox axis
    -->
      <subgroupParam>0.3 20.0 240.0</subgroupParam>
    </forecastClass>
</OceanForecast>
  
<!-- Output directory name-->
  <OutputDirectory>mygab</OutputDirectory>
</lecode:oceanInput>

# Running the coupled model

To run the model we need to gives the full path to:

1. the folder containing the input file: in this case <code>'./gab'</code>
2. the input file name: in this case <code>'gabocean.xml'</code>

The run takes about 200s to finish...

In [6]:
#myos.ocean_command('./gab','gabocean.xml')

From the input file we should have the run output stored in a folder name <code>mygab</code>. Using shell command list the files contained in this directory:

In [8]:
%ls
myos.list_dir('./gab/mygab')

bath.csv  gab/  Lecture-overview.ipynb  myOSshell.py  myOSshell.pyc  NB1-SWE2D.ipynb  NB2-BathymetryGen.ipynb  NB3-OceanSim.ipynb  SydOceanT/  XYZ_Index.jpg
[]


# Compressing the output and Dropbox upload

To visualise the output we will be using a visualisation software called **Paraview**. At the moment this package is not installed as an IPython library and we will need to transfert our simulation files to our local computer where we have installed Paraview.

To do so we will first compress our output folder:

In [9]:
myos.tar_command('gabsim.tar','./gab/mygab')
%ls

Then we will upload the compressed folder called **gabsim.tar** from the Amazon server to our **Dropbox** account:

Dropbox credentials:

+ name mars5001
+ surname usyd
+ mail tristan.salles@sydney.edu.au
+ password mars5001

From the Dropbox we can finally get the compressed file on our local machine! 

In [11]:
myos.dropbox_upload('.','gabsim.tar','.')

# Cleaning

Before going to the visualisation make a bit of cleaning on the server side

In [13]:
# Delete the output folder
myos.delete('/gab/mygab')

In [14]:
# Delete the compressed file
myos.delete('gabsim.tar')

# Visualisation with Paraview<img src='http://www.paraview.org/wp-content/uploads/2014/04/ParaViewLogo.png' width=100>


### What is Paraview

Paraview is an open-source, multi-platform data analysis and visualization application. ParaView users can quickly build visualizations to analyze their data using qualitative and quantitative techniques. The data exploration can be done interactively in 3D or programmatically using ParaView’s batch processing capabilities.

### Download Paraview


In [10]:
from IPython.core.display import HTML
HTML('<iframe src=http://www.paraview.org width=600 height=500></iframe>')

### Model output structure

From Dropbox you have downloaded **gabsim.tar** to your local machine. Once it's done uncompressed the tar file then navigate to the output directory called **mygab**, the model output structure should be similar to this:

+ mygab
    + outputs
        + Ocean_series.xdmf
        + Ocean.XXXX.xmf
        + OceanGrid.XXXX.h5
        + swan.csv
    + inputs
        + swanInfo
        + swan.bot
        + swan.swn

The only file that needs to be loaded in Paraview is the **Ocean_series.xdmf** which is a time serie file that is automatically calling the other files through **Paraview**.


### Paraview calculators

The model output records the following parameters:

+ Bathy: bathymetry
+ CurU: ocean circulation current along X
+ CurV: ocean circulation current along Y
+ WavU: wave induced current along X
+ WavV: wave induced current along Y
+ WavHs: significant wave height
+ WavP:  wave period
+ WavWl: wave lenght

To compute induced current magnitude you can use a **Calculator** in Paraview like this.

#### Ocean circulation current 

<code>iHat\*CurU_grp1+jHat\*CurV_grp1</code>

#### Wave current

<code>iHat\*WavU_grp1+jHat\*WavV_grp1</code>

#### Combined wave/ocean

<code>iHat\*(CurU_grp1+WavU_grp1) +jHat\*(CurV_grp1+WavV_grp1)</code>


### Loading state

In Dropbox, there is a folder called **ParaviewUtility** which contains a file called *pv_state.pvsm*. This file is a *paraview state* that contains a predefined visualisation state with calculators and glyphs already set up as well as color scale. You can download this state and try to open the state from Paraview. You will have to point to the directory where you've uncompressed the model output. Depending of your OS (Mac,Linux,Windows) it might not work perfectly....   
<br/>
<br/>
### Combined ocean circulation & wave velocity field for wind scenario 2

<img src='./gab/gab3.jpg' width=700>


# Example of impact of sea-level change on sediment transport

Below is an example of using the same dataset as yours plus the sediment transport component to look at long-term evolution of transport from land to coast and from coast to shelf:

#### Wind induced currents 
<img src='https://lh3.googleusercontent.com/-VMq7k4O7D90/U6_m8jyhgsI/AAAAAAAAARU/oJnqT__jKA8/w1012-h894/circFW.avi' width=200>

#### Surface mean grain size evolution
<img src='https://lh3.googleusercontent.com/-mEVX0SDc2Gg/U6_vUaq0jxI/AAAAAAAAATA/Lv6DWxu1AkQ/w1012-h894/gsz.avi' width=200>

#### Erosion deposition evolution
<img src='https://lh3.googleusercontent.com/-w9Q-MsWFEcI/U6_otZV0m7I/AAAAAAAAASM/yAIW2PJTV8U/w1012-h894/elevChg.avi' width=200>